
This is code posted here discussing the 80ms pauses that seem to happen for every block of 4096 bytes
https://forum.micropython.org/viewtopic.php?f=18&t=4725&p=27605#p27605

### See later cells for attempts at using hardware times

Stripdown version that just saves ticks time would be:

`fout = open("data5.txt", "w")
for i in range(n):
    t = time.ticks_ms()
    fout.write("%08d" % t)
fout.close()
`

`%fetchfile --binary data5.txt`

And on the PC to plot the dt of timestamps:

`from matplotlib import pyplot as plt
%matplotlib inline
k = open("data5.txt").read()
k = [ int(k[i-8:i])  for i in range(8, len(k), 8) ]
k = [a-b  for a,b in zip(k[1:], k)]
plt.plot(k)`


In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [2]:
import time

n = 3000
longdt = 20

fout = open("data5.txt", "w")
d = [ ]
t0 = time.ticks_ms()

sumdt = 0
i0 = 0
for i in range(n):
    t = time.ticks_ms()
    fout.write("%08d" % t)
    dt = t-t0
    if dt > longdt:
        d.append(((i-i0)*8, dt))
        i0 = i
        if len(d) > 20:
            break
    else:
        sumdt += dt
    t0 = t
fout.close()
t = time.ticks_ms()
print("closetime", t-t0)
if i == n-1:
    print("meanshortwrite", sumdt/(n-len(d)))
    print("longwrite_ms", d)
else:
    print(i, n)
    print(d)

closetime 99
meanshortwrite 1.337007
longwrite_ms [(8, 52), (4096, 98), (4096, 97), (4096, 98), (4096, 96), (4096, 97)]


### Hardware timing code based on this link using a ring buffer

https://techtutorialsx.com/2017/10/07/esp32-micropython-timer-interrupts/


In [56]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [58]:
# first the ring buffer
import array, time
nt = 40
pt = array.array("I", range(0,nt+2))
 # [i, tprev, dt, dt, ..., dt]
pt[1] = time.ticks_ms()
pt[0] = 0
def irqtrigger(p):
    t1 = time.ticks_ms()
    i = pt[0]+1
    pt[(i%nt)+2] = t1 # - pt[1]
    pt[1] = t1
    pt[0] = i


In [52]:
irqtrigger(1)
print(pt[(pt[0]%nt)+2], time.ticks_ms())

2104841 2104842


In [43]:
timer.deinit()
print(timer)

Timer(3ffb5550; alarm_en=1, auto_reload=1, counter_en=0)


In [59]:
from machine import Timer
timer = Timer(0)
timer.init(period=100, mode=Timer.PERIODIC, callback=irqtrigger)


In [66]:
timer.deinit()
timer.init(period=10, mode=Timer.PERIODIC, callback=irqtrigger)


In [71]:
#print(pt[(pt[0]%nt)+2], time.ticks_ms())
print(pt[0])


2073


In [74]:
import time
n = 3000
fout = open("data6.txt", "w")
j = pt[0]
for i in range(n):
    while j == pt[0]:
        time.sleep_ms(2)
    while j != pt[0]:
        j += 1
        fout.write("%08d" % pt[(j%nt)+2])
fout.close()


......

In [75]:
%fetchfile --binary data6.txt
# Looks just as bad in the plotting

Fetched 24336=24336 bytes from data6.txt.


In [ ]:
# self-
sumdt = 0
i0 = 0
for i in range(n):
    t = time.ticks_ms()
    fout.write("%08d" % t)
    dt = t-t0
    if dt > longdt:
        d.append(((i-i0)*8, dt))
        i0 = i
        if len(d) > 20:
            break
    else:
        sumdt += dt
    t0 = t
fout.close()
t = time.ticks_ms()
print("closetime", t-t0)
if i == n-1:
    print("meanshortwrite", sumdt/(n-len(d)))
    print("longwrite_ms", d)
else:
    print(i, n)
    print(d)